In [1]:
#coding:utf-8
from tensorflow.contrib.tensorboard.plugins import projector
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import scipy.io as sio

In [ ]:
learning_rate = 0.0001
batch_size=5
def read_data(filename):
    with open(filename,'rb') as f:
        #记载matlab文件
        dict = sio.loadmat(f)
    return dict['fea'],dict['gnd']

train_data,train_labels = read_data('YaleB_32x32.mat')
log_dir = 'yalebsample2'
name_to_visualise_variable = 'yalebembedding2'
#保存数据
path_for_face_png = os.path.join(log_dir,'newface.png')
path_for_face_data = os.path.join(log_dir,'newface.tsv')

#建立embedding
embedding_var = tf.Variable(train_data,name=name_to_visualise_variable)
#将信息写入log_dir目录下
summary_writer = tf.summary.FileWriter(log_dir)
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name
embedding.metadata_path = path_for_face_data
embedding.sprite.image_path = path_for_face_png
embedding.sprite.single_image_dim.extend([32,32])
#将embedding可视化
projector.visualize_embeddings(summary_writer,config)

sess1 = tf.InteractiveSession()
sess1.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.save(sess1,os.path.join(log_dir,'model.ckpt'),1)

#将图片拼成一张大图
def create_sprite_image(images):
    if isinstance(images, list):
        images = np.array(images)
    img_h = images.shape[1]
    img_w = images.shape[2]
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))


    spriteimage = np.ones((img_h * n_plots ,img_w * n_plots ))

    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                this_img = images[this_filter]
                spriteimage[i * img_h:(i + 1) * img_h,
                  j * img_w:(j + 1) * img_w] = this_img

    return spriteimage

#将矩阵转为图片
def vector_to_matrix_face(face_digits):
    return np.reshape(face_digits,(-1,32,32))

#将图片转为灰度

def invert_grayscale(face_digits):
    return 1-face_digits

to_visualise = train_data
to_visualise = vector_to_matrix_face(to_visualise)
to_visualise = to4d(to_visualise)

sprite_image = create_sprite_image(to_visualise)
plt.imsave(path_for_face_png,sprite_image,cmap='gray')
plt.imshow(sprite_image,cmap='gray')

with open(path_for_face_data,'w') as f:
    f.write('Index\tLabel\n')
    for index,label in enumerate(train_labels):
        f.write('%d\t%d\n' %(index,label))

#将标签转为0-37
train_labels = train_labels-1

#shuffle data
np.random.seed(100)
train_data = np.random.permutation(train_data)
np.random.seed(100)
train_labels = np.random.permutation(train_labels)
test_data = train_data[0:1690,:]
test_labels = train_labels[0:1690]
train_data = train_data[1691:,:]
train_labels = train_labels[1691:]
np.random.seed(200)
test_data = np.random.permutation(test_data)
np.random.seed(200)
test_labels = np.random.permutation(test_labels)

#将标签转为one_hot类型
def label_to_one_hot(labels_dense, num_classes=38):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

# #将图片转为灰度图
def to4d(img):
    return img.reshape(img.shape[0],32,32,1).astype(np.float32)/255
print(train_data.shape[0])
train_data = to4d(train_data)

train_labels = label_to_one_hot(train_labels,38)
test_data = to4d(test_data)
test_labels = label_to_one_hot(test_labels,38)

xs = tf.placeholder(tf.float32,[None,32,32,1])
ys = tf.placeholder(tf.float32,[None,38])
keep_prob = tf.placeholder(tf.float32)


#开始构建卷积神经网络
conv1 = tf.layers.conv2d(inputs=xs,filters=16,kernel_size=2,strides=1,padding='same',activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1,pool_size=2,strides=2,padding='same')
conv2 = tf.layers.conv2d(pool1,filters=36,kernel_size=2,strides=1,padding='same',activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2,pool_size=2,strides=2,padding='same')
flat = tf.reshape(pool2,[-1,8*8*36])
dense = tf.layers.dense(flat,512,tf.nn.relu)
dropout = tf.nn.dropout(dense,keep_prob)
output = tf.layers.dense(dropout,38)

#
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
#计算loss
loss = tf.losses.softmax_cross_entropy(onehot_labels=ys,logits=output)
tf.summary.scalar('loss',loss)
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)
#返回两个参数一个train_opt,一个acc
accuracy = tf.metrics.accuracy(labels=tf.argmax(ys,axis=1),predictions=tf.argmax(output,axis=1))[1]
tf.summary.scalar('accuracy',accuracy)
merged = tf.summary.merge_all()
with tf.Session() as sess:
    init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
    sess.run(init)
    for step in range(200):
        i = 0
        while i < len(train_data):
            start = i
            end = i+batch_size
            batch_x = np.array(train_data[start:end])
            batch_y = np.array(train_labels[start:end])
            _,c = sess.run([train,loss],feed_dict={xs:batch_x,ys:batch_y,keep_prob:0.75})
            i += batch_size
        if step % 1 ==0:
            acc = sess.run(accuracy,feed_dict={xs:test_data,ys:test_labels,keep_prob:1})
            print('= = = = = = > > > > > > ','step:',step,'loss: %.4f'%c,'accuracy: %.2f' %acc)
            summary,_ = sess.run([merged,train],feed_dict={xs:test_data,ys:test_labels,keep_prob:1})
            summary_writer.add_summary(summary,step)
           
